In [ ]:
import xml.etree.ElementTree as ET
import os

def xml_to_yolo(xml_path, image_width, image_height):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    yolo_lines = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        if class_name not in class_mapping:
            continue
        class_id = class_mapping[class_name]

        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)

        x_center = (xmin + xmax) / (2.0 * image_width)
        y_center = (ymin + ymax) / (2.0 * image_height)
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        yolo_line = f"{class_id} {x_center} {y_center} {width} {height}"
        yolo_lines.append(yolo_line)

    return yolo_lines

# Veri seti dizini
data_dir = "/Users/muratguc/Projects/SeniorProject/ObjectClassification/datasets/WOTR"

# Sınıf adları ve sınıf indeksleri
class_mapping = {
    'tree':0,
    'red_light':1,
    'green_light':2,
    'crosswalk':3,
    'tactile_paving':4,
    'sign':5, 
    'pedestrian':6, 
    'bicycle':7, 
    'bus':8, 
    'truck':9, 
    'car':10,
    'motorcycle':11, 
    'reflective_cone':12, 
    'ashcan':13, 
    'warning_column':14,
    'roadblock':15, 
    'pole':16, 
    'dog':17, 
    'tricycle':18,
    'fire_hydrant':19
}
for root, dirs, files in os.walk(os.path.join(data_dir, "Annotations/test")):
    for file in files:
        if file.endswith(".xml"):
            xml_path = os.path.join(root, file)
            image_name = os.path.splitext(file)[0]
            image_width, image_height = 876, 657  

            yolo_lines = xml_to_yolo(xml_path, image_width, image_height)

            with open(os.path.join(data_dir, "labels", f"{image_name}.txt"), "w") as txt_file:
                for line in yolo_lines:
                    txt_file.write(line + "\n")


In [ ]:
import os
import cv2

val_folder = '/Users/muratguc/Projects/SeniorProject/ObjectClassification/datasets/WOTR/test'  # Val klasörünün yolu
images_folder = os.path.join(val_folder, 'images') 
labels_folder = os.path.join(val_folder, 'labels') 


image_dimensions = {}
for filename in os.listdir(images_folder):
    if filename.endswith(('.jpg', '.png', '.jpeg')):  
        image_path = os.path.join(images_folder, filename)
        image = cv2.imread(image_path)
        if image is not None:
            image_height, image_width, _ = image.shape
            image_dimensions[filename] = (image_width, image_height)

for filename in os.listdir(labels_folder):
    if filename.endswith('.txt'):  
        label_path = os.path.join(labels_folder, filename)
        
        with open(label_path, 'r') as file:
            lines = file.readlines()
        
        normalized_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5: 
                class_id, x_center, y_center, width, height = parts
                if filename in image_dimensions:
                    image_width, image_height = image_dimensions[filename]
                    x_center = float(x_center) / image_width
                    y_center = float(y_center) / image_height
                    width = float(width) / image_width
                    height = float(height) / image_height
                normalized_line = f"{class_id} {x_center} {y_center} {width} {height}\n"
                normalized_lines.append(normalized_line)
        
        with open(label_path, 'w') as file:
            file.writelines(normalized_lines)

print("Tüm label dosyaları normalleştirildi.")


In [ ]:
import cv2
import os

image_path = '/Users/muratguc/Projects/SeniorProject/ObjectClassification/datasets/WOTR/train/images/10013902.jpg'
label_path = '/Users/muratguc/Projects/SeniorProject/ObjectClassification/datasets/WOTR/train/labels/10013902.txt'

image = cv2.imread(image_path)

image_height, image_width, _ = image.shape

with open(label_path, 'r') as file:
    lines = file.readlines()

for line in lines:
    parts = line.strip().split()
    if len(parts) == 5:
        class_id, x_center, y_center, width, height = map(float, parts)
        
        x_center = int(x_center * image_width)
        y_center = int(y_center * image_height)
        width = int(width * image_width)
        height = int(height * image_height)
        
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)
        
        color = (0, 255, 0)  
        thickness = 2 
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)

cv2.imshow('Annotated Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import os

labels_folder = '/Users/muratguc/Projects/SeniorProject/ObjectClassification/datasets/WOTR/train/labels/'  # Labels klasörünün yolu
count_non_normalized = 0 

for filename in os.listdir(labels_folder):
    if filename.endswith('.txt'):
        label_path = os.path.join(labels_folder, filename)

        with open(label_path, 'r') as file:
            lines = file.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5: 
                class_id, x_center, y_center, width, height = map(float, parts)
                if x_center < 0 or x_center > 1 or y_center < 0 or y_center > 1 or \
                   width < 0 or width > 1 or height < 0 or height > 1:
                    count_non_normalized += 1

print(f"Toplam {count_non_normalized} dosya non-normalized değerler içeriyor.")


Toplam 7908 dosya non-normalized değerler içeriyor.


In [5]:
import os

labels_folder = '/Users/muratguc/Projects/SeniorProject/ObjectClassification/datasets/WOTR/val/labels/'  # Labels klasörünün yolu
images_folder = '/Users/muratguc/Projects/SeniorProject/ObjectClassification/datasets/WOTR/val/images/'  # Images klasörünün yolu

for filename in os.listdir(labels_folder):
    if filename.endswith('.txt'): 
        label_path = os.path.join(labels_folder, filename)

        with open(label_path, 'r') as file:
            lines = file.readlines()

        non_normalized = False
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5:  
                class_id, x_center, y_center, width, height = map(float, parts)
                if x_center < 0 or x_center > 1 or y_center < 0 or y_center > 1 or \
                   width < 0 or width > 1 or height < 0 or height > 1:
                    non_normalized = True
                    break

        if non_normalized:
            image_filename = os.path.splitext(filename)[0] + '.jpg'  # Görüntü dosyasının adını oluştur
            image_path = os.path.join(images_folder, image_filename)

            os.remove(label_path)
            os.remove(image_path)

print("Non-normalized değerler içeren dosyalar kaldırıldı.")


Non-normalized değerler içeren dosyalar kaldırıldı.


In [ ]:
import os
import cv2

folder_path = "/Users/muratguc/Projects/SeniorProject/ObjectClassification/datasets/WOTR/test/images"

dimensions = []

for filename in os.listdir(folder_path):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        file_path = os.path.join(folder_path, filename)
        image = cv2.imread(file_path)
        if image is not None:
            height, width, _ = image.shape
            dimensions.append((width, height))

print(dimensions)

In [15]:
import os
import shutil
import xml.etree.ElementTree as ET

kaynak_klasor = "/Users/muratguc/Downloads/WOTR/Annotations"
hedef_klasor = "/Users/muratguc/Projects/SeniorProject/ObjectClassification/datasets/train_tactile_paving_annotations"

if os.path.exists(hedef_klasor):
    shutil.rmtree(hedef_klasor)
os.makedirs(hedef_klasor)

for dosya_adı in os.listdir(kaynak_klasor):
    if dosya_adı.endswith(".xml"):
        xml_dosya_path = os.path.join(kaynak_klasor, dosya_adı)

        try:
            tree = ET.parse(xml_dosya_path)
            root = tree.getroot()

            for object_etiket in root.findall("object"):
                name_etiket = object_etiket.find("name")
                if name_etiket is not None and name_etiket.text == "tactile_paving":
                    # Eşleşen XML dosyasını hedef klasöre kopyala
                    hedef_dosya_path = os.path.join(hedef_klasor, dosya_adı)
                    shutil.copy(xml_dosya_path, hedef_dosya_path)
                    break
        except ET.ParseError:
            print(f"{xml_dosya_path} dosyası geçersiz XML formatına sahip.")

print("İşlem tamamlandı.")


İşlem tamamlandı.


In [3]:
import os

# Dizin yolunu belirtin
dizin_yolu = "/Users/muratguc/Projects/SeniorProject/ObjectClassification/v2/tactile paving/valid/labels"

# Sınıf endekslerini güncelleyecek fonksiyon
def sınıf_endekslerini_güncelle(dosya_yolu, güncelleme_eşlemesi):
    try:
        with open(dosya_yolu, "r") as dosya:
            icerik = dosya.readlines()

        güncellenmiş_icerik = []
        for satır in icerik:
            satır_parçaları = satır.strip().split()
            if len(satır_parçaları) > 0:
                eski_sınıf_endeksi = int(satır_parçaları[0])
                if eski_sınıf_endeksi in güncelleme_eşlemesi:
                    yeni_sınıf_endeksi = güncelleme_eşlemesi[eski_sınıf_endeksi]
                    satır_parçaları[0] = str(yeni_sınıf_endeksi)
                güncellenmiş_satır = " ".join(satır_parçaları)
                güncellenmiş_icerik.append(güncellenmiş_satır)

        with open(dosya_yolu, "w") as dosya:
            dosya.write("\n".join(güncellenmiş_icerik))

    except Exception as e:
        print(f"{dosya_yolu} dosyası güncellenirken bir hata oluştu: {str(e)}")

# Sınıf endekslerini güncellemek için eşlemeyi tanımlayın
sınıf_endeks_eşlemesi = {0: 4}   # Örnek eşleme

# Dizin içindeki tüm dosyaları dolaşın ve sınıf endekslerini güncelleyin
for dosya_adı in os.listdir(dizin_yolu):
    dosya_yolu = os.path.join(dizin_yolu, dosya_adı)

    if os.path.isfile(dosya_yolu):
        sınıf_endekslerini_güncelle(dosya_yolu, sınıf_endeks_eşlemesi)

print("Sınıf endeksleri güncellendi.")


Sınıf endeksleri güncellendi.
